# torch to ONNX

In [1]:
import torch
import numpy as np

def to_onnx(model, input_shape, dynamic_shape=True, onnx_path="model.onnx"):
    model.eval()
    dummy_input = torch.ones(*input_shape, dtype=torch.float32, device=cuda0)

    input_names=['input']
    output_names=['output']
    if dynamic_shape:
        dynamic_axes= {'input': {0:'batch_size'}, 'output': {0:'batch_size'}} # define batch_size as dynamic input
    else:
        dynamic_axes = None
    
    torch.onnx.export(model, dummy_input, onnx_path, input_names=input_names, output_names=output_names, dynamic_axes=dynamic_axes)

### Bypass model

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
cuda0 = torch.device('cuda:0')

batch_size = 1
height = 256
width = 128

input_shape = (batch_size, 3, height, width)

class BypassNet(torch.nn.Module):
    def __init__(self, num_clasess=512):
        super().__init__()
        self.num_clasess = torch.tensor(num_clasess, device=cuda0)
        vector = np.zeros(shape=num_clasess, dtype=np.float32)
        self.vector = torch.from_numpy(vector).to(cuda0)
    def forward(self, input):
        

        x1 = torch.flatten(input)[:128]
        x2 = torch.flatten(input)[32768:(32768+128)]
        x3 = torch.flatten(input)[65536:(65536+128)]
        x4 = torch.flatten(input)[128:256]

        out = torch.cat((x1, x2, x3, x4), 0)
        out = torch.reshape(out, (1, self.num_clasess))
        print("out shape", out.shape)
        return out

model = BypassNet(num_clasess=512)
to_onnx(model, input_shape, False)

out shape torch.Size([1, 512])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
